This is an attempt in doing exploratory analysis of the data for the
Personalized Medicine: Redefining Cancer Treatment .I have used inputs from another kernel https://www.kaggle.com/headsortails/personalised-medicine-eda-with-tidy-r.

In [ ]:
from sklearn import *
import sklearn
import pandas as pd
import numpy as np
import sys
train = pd.read_csv('../input/training_variants')
test = pd.read_csv('../input/test_variants')
train_text = pd.read_csv('../input/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text = pd.read_csv('../input/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

print('train',train.shape)
print('test',test.shape)
total_unique_gene=len(train.Gene.unique())+len(test.Gene.unique())
total_unique_variation=len(train.Variation.unique())+len(test.Variation.unique())
print("*"*100)

print('Test data has ',100*(test.shape[0]-train.shape[0])/train.shape[0],'% more data')
print("*"*100)

print('Count of unique train Genes',len(train.Gene.unique()))
print('Count of unique train Variation',len(train.Variation.unique()))
print("*"*100)

print('Count of unique test Genes',len(test.Gene.unique()))
print('Count of unique test Variation',len(test.Variation.unique()))

print("*"*100)
print("Number of unique genes in the train and test dataset" ,total_unique_gene)
print("Number of unique variation in the train and test dataset" ,total_unique_variation)
print("*"*100)

print('Percentage of genes that are in the train set but not in test set and viceversa',
      len(set(train.Gene.unique()).symmetric_difference(set(test.Gene.unique())))*100/total_unique_gene)
print('Percentage of variation that are in the train set but not in test set and viceversa',
      len(set(train.Variation.unique()).symmetric_difference(set(test.Variation.unique())))*100/total_unique_variation)

print("*"*100)

print('Intersection of train and test Genes',len(set(train.Gene.unique()).intersection(set(test.Gene.unique())))*100/total_unique_gene)

print('Intersection of train and test Variation',len(set(train.Variation.unique()).intersection(set(test.Variation.unique())))*100/total_unique_variation)


# Observations
* Train has 3321 points and test has 5668 so about 70% more data
* 264 unique genes in train set and 2996 unique variation in train set
* 1397 unique genes in test set and 5628 unique variation in test set
* Total unique genes 1661
* Total unique variations 8624

* Overlap of train and test genes is only 9.7% (OUCH!!!!)
* Overlap of train and test variation is 0.17% (BIG OUCH!!!!)



# Observation
* Text value is null for 5 rows in train set

In [ ]:
print('Text values with null values in training set',train_text.loc[train_text.Text=='null','Text'].shape[0])
print('Text values with null values in testing  set',test_text.loc[test_text.Text=='null','Text'].shape[0])


In [ ]:
train.isnull().sum()

In [ ]:
train.groupby('Gene').ID.count().sort_values(ascending=False).head()

In [ ]:
test.groupby('Gene').ID.count().sort_values(ascending=False).head()

In [ ]:
train.groupby('Variation').ID.count().sort_values(ascending=False).head()

In [ ]:
test.groupby('Variation').ID.count().sort_values(ascending=False).head()

In [ ]:
%matplotlib inline
df=(train.Gene.value_counts(normalize=True)*100)
df[df>1].plot(kind='barh',title='% of GENES in training set')

In [ ]:
df=(test.Gene.value_counts(normalize=True)*100)
df[df>=0.5].plot(kind='barh',title='% of GENES in test set')

In [ ]:
num_common_genes=set(train.Gene).intersection(set(test.Gene))

print( 'Common genes in train and test {0}'.format(len(num_common_genes)))


In [ ]:
train.Class.value_counts(normalize=True).plot(kind='bar',title='Class distribution in train set')

Observation

  * Class levels 3, 8, and 9 are notably under-represented

  * Levels 5 and 6 are of comparable, medium-low frequency

  * Levels 1, 2, and 4 are of comparable, medium-high frequency

  * Level 7 is clearly the most frequent one


# Feature interaction

## Gene length vs Class

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,5)

train['Gene_Length']=train.Gene.apply(lambda x:len(x))
pd.crosstab(train.Class,train.Gene_Length).plot(kind='bar',title='Class distribution versus Gene length')

Observations
* Class 2 does not have genelength of 8
* Class 3,4 does not have genelength of 2 and 8
* Class 5 does not have gene length of 7,8
* Class 6,7 goes not have gene length 8
* Class 8 does not have genes with length 2,3,7,8
* Class 9 does not have genes with length 2,6,7,8

# Gene vs Class

In [ ]:
train_top=train.Gene.value_counts(normalize=True)*100
top_genes=train_top[train_top>1].index
uq=train.Class.unique()
train_top_genes=train[train.Gene.isin(top_genes)]
topGenes=train_top_genes.Gene.unique()

test_top_genes=test[test.Gene.isin(top_genes)]


In [ ]:
print(train_top_genes.Gene.unique())
print(test_top_genes.Gene.unique())

# Observation
***Only 2 genes that occur more than 1% in test dataset


In [ ]:
#For every class find out what is the most predominant genes
plt.rcParams["figure.figsize"] = (20,10)
import pylab
row=0
col=0
fig, axes = plt.subplots(nrows=3, ncols=3)
#pylab.gca().get_ylabel().set_fontsize(10)
for i in range(0,len(uq)):
    ax1=axes[row,col]
    train_top_genes[train_top_genes.Class==uq[i]].Gene.value_counts().reindex(topGenes).plot(kind='bar',ax=ax1,title=uq[i])
    plt.subplots_adjust(top=1.3)
    ax1.set_yticks(np.arange(0,105, 5))

    col=col+1
    if(col==3):
        col=0
        row=row+1
        
        

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
fig, axes = plt.subplots(nrows=11, ncols=2)

uq_genes=train_top_genes.Gene.unique()
row=0
col=0
for i in range(0,len(uq_genes)):
    ax1=axes[row,col]

    train_top_genes[train_top_genes.Gene==uq_genes[i]].Class.value_counts().reindex(uq).plot(kind='bar',ax=ax1,title=uq_genes[i])
    col=col+1
    if(col==2):
        col=0
        row=row+1
    plt.subplots_adjust(top=2.5)

        



# Observation

* Class 8 and Class 9 dont have the top genes


# Gene versus variation

In [ ]:
from sklearn import preprocessing 
train_top_genes['sod']='train'
test_top_genes['sod']='test'

df1=train_top_genes.loc[:,['ID','Gene','Variation','sod']].append(test_top_genes)
le = preprocessing.LabelEncoder()
var=le.fit_transform(df1.Variation)
le = preprocessing.LabelEncoder()
var=le.fit_transform(df1.Variation)
df1['Int_var']=var
df1.head()

In [ ]:
train_top_genes=df1[df1.sod=='train'].sort_values(by='Gene')
test_top_genes=df1[df1.sod=='test'].sort_values(by='Gene')
df=train_top_genes.groupby(['Gene','Int_var'],as_index=False).ID.count()
df.head()

In [ ]:
import seaborn as sns
sns.set(color_codes=True)
plt.rcParams["figure.figsize"] = (20,5)

sns.stripplot(x="Gene", y="Int_var", data=train_top_genes)
sns.plt.suptitle('Train title versus variation ')


In [ ]:
sns.stripplot(x="Gene", y="Int_var", data=test_top_genes)
sns.plt.suptitle('Test title versus variation ')



# Observation 
* Train and test data sets plot of variation vs gene is very different indicating different datasets.

# Text data

In [ ]:
train=train.merge(train_text)
test=test.merge(test_text)
print(train.shape)
print(test.shape)
train.Text.head()

Text seems to be from kinda screen scraping algorithim(Previous SectionNext SectionResults)

* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3845122/
* http://www.humangenetik-freiburg.de/mediapool/43/433039/data/FAM58ANatGenet2008.pdf

In [ ]:
train.iloc[0].Text

In [ ]:
train['length_str']=train.Text.apply(lambda x:len(x))
test['length_str']=test.Text.apply(lambda x:len(x))
train['word_count_str']=train.Text.apply(lambda x:len(x.split()))
test['word_count_str']=test.Text.apply(lambda x:len(x.split()))
train.head()

In [ ]:

sns.distplot(test.word_count_str,label='test')
sns.distplot(train.word_count_str,label='train')
plt.legend(loc='upper right')

# Observation 
* Distribution of text length  between train and tedt is different .

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

train.groupby('Class').word_count_str.median().plot(kind='bar',title='Word_Count by class')

# Observation
* Class 8 has text with most words and class 3 has least

In [ ]:
#For every class find out what is the most predominant genes
plt.rcParams["figure.figsize"] = (20,10)
import pylab
row=0
col=0
fig, axes = plt.subplots(nrows=3, ncols=3)

#pylab.gca().get_ylabel().set_fontsize(10)
for i in range(0,len(uq)):
    ax1=axes[row,col]
    ax1.set_ylim(0,0.000175)
    ax1.set_title('Class '+str(i), fontsize =16)
    sns.kdeplot(train.loc[train.Class==uq[i],'word_count_str'],ax=ax1,shade=True, color="r")

    plt.subplots_adjust(top=1.3)

    col=col+1
    if(col==3):
        col=0
        row=row+1
        
        

In [ ]:
test.word_count_str.describe()

In [ ]:
train.word_count_str.describe()

In [ ]:

#Packages Imports
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from os import path
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

from nltk.tokenize import sent_tokenize

from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')


train['txt']=train.Text.apply(lambda x:' '.join(i for i in x.lower().split() if i not in stop))


In [ ]:

df=train.groupby('Class').txt.sum()
df.head()

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,10)
import pylab

#pylab.gca().get_ylabel().set_fontsize(10)
for i in range(0,len(uq)):
    wordcloud =WordCloud(background_color='black',  max_font_size=50, max_words=100).generate(df.loc[uq[i]])
    plt.imshow(wordcloud)
    plt.title('Class '+str(i))
    plt.axis("off")
    plt.show()


    

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
import pylab
row=0
col=0
fig, axes = plt.subplots(nrows=3, ncols=3)

#pylab.gca().get_ylabel().set_fontsize(10)
for i in range(0,len(uq)):
    ax1=axes[row,col]
    ax1.set_ylim(0,70000)
    ax1.set_title('Class '+str(i), fontsize =16)


    pd.Series(pd.Series(df.loc[uq[i]]).str.split()[0]).value_counts().sort_values(ascending=False)[:50].plot(kind='barh',ax=ax1)
    plt.subplots_adjust(top=3)

    col=col+1
    if(col==3):
        col=0
        row=row+1

#TEST MINING MORE TO COME